In [ ]:
from google.colab import drive


In [ ]:
%cd /content/drive/MyDrive/CV_Final/Final/

/content/drive/MyDrive/CV_Final/Final


In [ ]:
import cv2
import dlib
import time
import math

In [ ]:
!python yolo_video_new.py --input videos/1.MP4 --output output/demo.avi --yolo yolo-coco

# **Cascade Classifier**

In [ ]:
carCascade = cv2.CascadeClassifier('yolo-coco/vech.xml')
video = cv2.VideoCapture('output/demo.avi')

WIDTH = 1280
HEIGHT = 720

def estimateSpeed(location1, location2):
    d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
    # ppm = location2[2] / carWidht
    ppm = 150
    d_meters = d_pixels / ppm
    fps = 7
    speed = d_meters * fps * 3.6*5.1
    round(speed, 2)
    return speed

In [ ]:
def trackMultipleObjects():
    rectangleColor = (0, 255, 0)
    frameCounter = 0
    currentCarID = 0
    fps = 0

    carTracker = {}
    carNumbers = {}
    carLocation1 = {}
    carLocation2 = {}
    speed = [None] * 1000

    out = cv2.VideoWriter('output/final_demo.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (WIDTH, HEIGHT))

    while True:
        # print(1)
        start_time = time.time()
        rc, image = video.read()
        if type(image) == type(None):
            # print(11)
            break

        image = cv2.resize(image, (WIDTH, HEIGHT))
        resultImage = image.copy()

        frameCounter = frameCounter + 1
        print("Frame no. ",frameCounter)
        carIDtoDelete = []
        # if frameCounter==120:
        #   # print(2)
        #   break
        
        for carID in carTracker.keys():
            trackingQuality = carTracker[carID].update(image)
            # print(trackingQuality)
            if trackingQuality < 3:
                carIDtoDelete.append(carID)

        
        for carID in carIDtoDelete:
            # print("Removing carID " + str(carID) + ' from list of trackers. ')
            # print("Removing carID " + str(carID) + ' previous location. ')
            # print("Removing carID " + str(carID) + ' current location. ')
            carTracker.pop(carID, None)
            carLocation1.pop(carID, None)
            carLocation2.pop(carID, None)

        
        if not (frameCounter % 10):
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24))

            for (_x, _y, _w, _h) in cars:
                x = int(_x)
                y = int(_y)
                w = int(_w)
                h = int(_h)

                x_bar = x + 0.5 * w
                y_bar = y + 0.5 * h

                matchCarID = None

                for carID in carTracker.keys():
                    trackedPosition = carTracker[carID].get_position()

                    t_x = int(trackedPosition.left())
                    t_y = int(trackedPosition.top())
                    t_w = int(trackedPosition.width())
                    t_h = int(trackedPosition.height())

                    t_x_bar = t_x + 0.5 * t_w
                    t_y_bar = t_y + 0.5 * t_h

                    if ((t_x <= x_bar <= (t_x + t_w)) and (t_y <= y_bar <= (t_y + t_h)) and (x <= t_x_bar <= (x + w)) and (y <= t_y_bar <= (y + h))):
                        matchCarID = carID

                if matchCarID is None:
                    # print(' Creating new tracker' + str(currentCarID))

                    tracker = dlib.correlation_tracker()
                    tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))

                    carTracker[currentCarID] = tracker
                    carLocation1[currentCarID] = [x, y, w, h]

                    currentCarID = currentCarID + 1

        for carID in carTracker.keys():
            trackedPosition = carTracker[carID].get_position()

            t_x = int(trackedPosition.left())
            t_y = int(trackedPosition.top())
            t_w = int(trackedPosition.width())
            t_h = int(trackedPosition.height())

            # cv2.rectangle(resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4)

            carLocation2[carID] = [t_x, t_y, t_w, t_h]

        end_time = time.time()

        if not (end_time == start_time):
            fps = 1.0/(end_time - start_time)

        for i in carLocation1.keys():
            if frameCounter % 1 == 0:
                [x1, y1, w1, h1] = carLocation1[i]
                [x2, y2, w2, h2] = carLocation2[i]

                carLocation1[i] = [x2, y2, w2, h2]

                if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                    # if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                    speedy=0
                    if speed[i]!=None:
                      speedy=speed[i]
                    speed[i] = (speedy+estimateSpeed([(x1+x1+w1)/2, (y1+y1+h1)/2], [(x2+x2+w2)/2, (y2+y2+h2)/2]))/2
                #    print(y1)
                    if speed[i] != None :
                        color=[0,0,255]
                        if speed[i]<49:
                          color=[ 0, 48, 143]
                        text = "{}: {}".format(round(speed[i],2) , "km/h")
                        # cv2.rectangle(resultImage, (x2,y2), (x2 + w2, y2 + h2), color, 4)
                        cv2.putText(resultImage, text, (int(x1 + w1/2), int(y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color ,2)
                        # cv2.putText(resultImage, speed[i] + "km/h", (int(x1 + w1/2), int(y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 100) ,2)

        # cv2.imshow('result', resultImage)

        out.write(resultImage)
        cv2.imwrite('yoloFrames/frame_'+str(frameCounter)+'.jpg',resultImage)

        

    
    cv2.destroyAllWindows()
    out.release()


trackMultipleObjects()

# **Speed Detection With Yolo**

In [ ]:
import numpy as np
import time
import cv2
import imutils
from imutils.video import FPS
from imutils.video import VideoStream
from google.colab.patches import cv2_imshow
import math

In [ ]:
def estimateSpeed(x1,y1,x2,y2):
    # print(x1,y1,x2,y2)
    d_pixels = math.sqrt(math.pow(abs(x2-x1), 2) + math.pow(abs(y2-y1), 2))
    # ppm = location2[2] / carWidht
    ppm = 17
    d_meters = d_pixels / ppm
    d_meters = d_meters/10
    fps = 7
    speed = d_meters * fps * 1.6
    return speed

In [ ]:
INPUT_FILE='videos/carsVid.mp4'
OUTPUT_FILE='output/speed_detection.avi'
LABELS_FILE='yolo-coco/coco.names'
CONFIG_FILE='yolo-coco/yolov3.cfg'
WEIGHTS_FILE='yolo-coco/yolov3.weights'
CONFIDENCE_THRESHOLD=0.3

H=None
W=None



LABELS = open(LABELS_FILE).read().strip().split("\n")

np.random.seed(4)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")


net = cv2.dnn.readNetFromDarknet(CONFIG_FILE, WEIGHTS_FILE)

vs = cv2.VideoCapture(INPUT_FILE)

ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
cnt =0
bigger_box=[]
out = cv2.VideoWriter('output/speed_detection.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (800, 600))
while True:
	cnt+=1
	print ("Frame number", cnt)
	try:
		(grabbed, image) = vs.read()
	except:
		break
	blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	net.setInput(blob)
	if W is None or H is None:
		(H, W) = image.shape[:2]
	layerOutputs = net.forward(ln)




	boxes = []
	confidences = []
	classIDs = []

	
	for output in layerOutputs:
		for detection in output:
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			if confidence > CONFIDENCE_THRESHOLD:
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))
				boxes.append([x, y, int(width), int(height)])
				confidences.append(float(confidence))
				classIDs.append(classID)


	idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD,
		CONFIDENCE_THRESHOLD)

	if len(idxs) > 0:
		for i in idxs.flatten():
			z=0
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])
			(x1, y1) = (boxes[i][0], boxes[i][1])
			(w1, h1) = (boxes[i][2], boxes[i][3])
			c1=0
			prevSpeed=0
			z1=1000000000
			for sett in bigger_box:
				x3=sett[0]
				y3=sett[1]
				w3=sett[2]
				h3=sett[3]
				c3=sett[4]

				z2=abs(x3-x)+abs(y3-y)+abs(w3-w)+abs(h3-h)+abs(confidences[i]-c3)
				if z2<z1:
					x1=x3
					y1=y3
					w1=w3
					h1=h3
					c1=c3
					prevSpeed=sett[5]
			x5=(x+x+w)/2
			x6=(x1+x1+w1)/2
			y5=(y+y+h)/2
			y6=(y1+y1+h1)/2
			z=estimateSpeed(x5,y5,x6,y6)
			if prevSpeed==0:
				prevSpeed=z
			z=(z+prevSpeed)/2
			color=[0,0,255]

			if z<49:
				color=[46, 139, 87]

			cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
			text = "{}: {:.2f}".format(LABELS[classIDs[i]], z)
			cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
				0.5, color, 2)


	out.write(cv2.resize(image,(800, 600)))
	cv2.imwrite('yoloFrames/frame_'+str(cnt)+'.jpg',cv2.resize(image,(800, 600)))
	key = cv2.waitKey(1) & 0xFF
	if key == ord("q"):
		break
	for values in boxes:
		newlist=[values[0],values[1],values[2],values[3],confidences[i],z]
		bigger_box.append(newlist)
	if cnt==170:
		break


cv2.destroyAllWindows()
out.release()

# **Non DL Speed Detection**

In [ ]:
import numpy as np
import cv2
import os

# generating image from video file
vidObj = cv2.VideoCapture('videos/sd3.webm')
fps = vidObj.get(cv2.CAP_PROP_FPS)
success = 1
success, img = vidObj.read()
#
# (success)
# fps = vidObj.get(cv2.CAP_PROP_FPS)
count = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT)) + 1

bs = np.zeros((count - 1, img.shape[0], img.shape[1], img.shape[2]), dtype = np.uint8)
bs1 = np.zeros((count - 1, img.shape[0], img.shape[1], img.shape[2]), dtype = np.uint8)
count = 1

while success:
    success, image = vidObj.read()

    if success:
        print(success)
        bs1[count - 1] = image
        kernel = np.ones((5, 5), np.float32) / 25
        image = np.asarray(cv2.filter2D(image, -1, kernel))
        bs[count - 1] = image
        count += 1

bgs = np.median(bs, 0)

# for loop to detect walking man in each frame
start = [-1, -1]
end = [-1, -1]
start_fame = -1
end_frame = -1
rds = 0
height, width, layers = bgs.shape
video_name = 'output/non_dl_speed.avi'
video = cv2.VideoWriter(video_name, 0, fps, (width, height))
for i in range(1, count):
    # reading each frame
    im = np.double(np.asarray(bs[i - 1]))
    img = np.double(np.asarray(bs1[i - 1]))
    kernel = np.ones((5, 5), np.float32) / 25
    im = cv2.filter2D(im, -1, kernel)

    # subtracting frame from backgroung and taking mean of r,b,g values
    fg = np.abs(im - bgs).mean(2)
    # assigning x1,x2,y1,y2 to coordinates in which object is present in current frame
    x1 = 0
    y1 = 0
    x2 = 0
    y2 = 0

    # for loop to find x1
    thrs = 80
    for ii in range(fg.shape[0]):
        found = False
        for j in range(fg.shape[1]):
            if (fg[ii][j] > thrs):
                print("yes")
                x1 = ii
                found = True
                break;
        if found:
            break;

    # for loop to find x2
    for ii in range(x1 + 1, fg.shape[0]):
        found = False
        for j in range(fg.shape[1]):
            if (fg[ii][j] > thrs):
                x2 = ii
                break;

    # for loop to find y1
    for ii in range(fg.shape[1]):
        found = False
        for j in range(fg.shape[0]):
            if fg[j][ii] > thrs:
                y1 = ii
                found = True
                break;
        if found:
            break;
    # for loop to find y2
    for ii in range(y1 + 1, fg.shape[1]):
        found = False
        for j in range(fg.shape[0]):
            if fg[j][ii] > thrs:
                y2 = ii
                break;

    # radius of circle
    r = max(abs(x2 - x1), abs(y2 - y1)) / 2
    rds = max(r, rds)
    print(r, rds)
    print(i)

    rx, ry = int(x1 + (x2 - x1) / 2), int(y1 + (y2 - y1) / 2)
    r = int(r)
    if 20 < r < 80 and rx >= fg.shape[0] * 0.33 and ry >= fg.shape[1] * 0.33:
        img = cv2.circle(img, (ry, rx), r, (0, 0, 255), 5)
        if start_fame == -1 or ry < end[1]-2:
            start = [rx, ry]
            start_fame = i
        end = [rx, ry]
        end_frame = i

    if start_fame > 0:
        distance = ((start[0] - end[0]) ** 2 + (start[1] - end[1]) ** 2) ** 0.5
        # print(fps)
        time = end_frame - start_fame
        speed = '0'
        if time > 0:
            speed = str(distance * 10 / time)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img,
                        "Speed is : " + speed,
                        (50, 50),
                        font, 1,
                        (0, 255, 255),
                        2,
                        cv2.LINE_4)
    img = img.astype(np.uint8)
    video.write(img)

video.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
